<h2>Summary of project</h2>
<br/>
<a href="https://www.kaggle.com/c/talkingdata-adtracking-fraud-detection)" target="_blank">Link to Kaggle Competition</a>

## Common imports

In [9]:
import pandas as pd
import numpy as np
import gc
import os, sys
import time
from sklearn.metrics import roc_auc_score

## Aisimplekit helpers

In [10]:
try:
    ## Occasionally (dev purpose only)
    sys.path.insert(0, "../..")
    import aisimplekit
except ModuleNotFoundError as err:
    print("""[err] {err}""".format(err=err))
    print("""Try: `pip install aisimplekit`""")

In [11]:
from aisimplekit.features.stats import *
from aisimplekit.models.lgb import lgb_train_cv
from aisimplekit.models.xgb import xgb_train_cv

## Custom notebook-specific helpers

In [12]:
def load_datasets(frm, to, debug=True, test_ofst=0, dtypes={}):
    """ """
    print('Loading train: #%d' % (to-frm))
    df_train = pd.read_csv(TRAIN_PATH, parse_dates=['click_time'], skiprows=range(1,frm), nrows=to-frm, dtype=dtypes, usecols=['ip','app','device','os', 'channel', 'click_time', 'is_attributed'])
    if debug != 0:
        nrows = 100000
        print('Loading test: #%d' % nrows)
        test_df = pd.read_csv(TEST_PATH, skiprows=range(1,1+test_ofst), nrows=nrows, parse_dates=['click_time'], dtype=dtypes, usecols=['ip','app','device','os', 'channel', 'click_time', 'click_id'])        
    elif debug == 0:
        print('Loading test: all. Param test_ofst ignored.')
        test_df = pd.read_csv(TEST_PATH, parse_dates=['click_time'], dtype=dtypes, usecols=['ip','app','device','os', 'channel', 'click_time', 'click_id'])
    else: raise

        # copy reference field
    sub = pd.DataFrame()
    sub['click_id'] = test_df['click_id'].astype('int')
    len_train = len(df_train)
    df_train = df_train.append(test_df)
    return (df_train, test_df, len_train, sub)

In [13]:
def extract_stats_features(df_train, predictors=['app', 'device', 'os', 'channel', 'hour', 'day']):
    """ """
    print('Extracting stats features...')
    df_train = do_countuniq( df_train, ['ip'], 'app', 'ip_uniq_app_count', 'uint8', show_max=True )
    df_train = do_countuniq( df_train, ['ip'], 'channel', 'ip_uniq_chan_count', 'uint8', show_max=True )
    df_train = do_count( df_train, ['ip', 'app'], 'ip_app_count', show_max=True )
    gc.collect()

    predictors.extend([col for col in df_train.columns if col.startswith('X')])
    predictors.extend([col for col in df_train.columns if col.startswith('Z')])

    if 'ip_tcount' in df_train.columns:
        predictors.extend(['ip_tcount'])
    if 'ip_app_count' in df_train.columns:
        predictors.extend(['ip_app_count'])
    if 'ip_app_os_count' in df_train.columns:
        predictors.extend(['ip_app_os_count'])
    if 'ip_tchan_count' in df_train.columns:
        predictors.extend(['ip_tchan_count'])
    if 'ip_app_os_var' in df_train.columns:
        predictors.extend(['ip_app_os_var'])
    if 'ip_app_channel_var_day' in df_train.columns:
        predictors.extend(['ip_app_channel_var_day'])
    if 'ip_app_channel_mean_hour' in df_train.columns:
        predictors.extend(['ip_app_channel_mean_hour'])

    return df_train, predictors

In [14]:
def extract_timeserie_features(df_train, predictors=['app', 'device', 'os', 'channel', 'hour', 'day']):
    """ """
    ## Timeseries features
    print('Extracting Timserie features...')

    print('[1/2] Extracting nextClick')
    D=2**26
    df_train['category'] = (df_train['ip'].astype(str) + "_" + df_train['app'].astype(str) + "_" + df_train['device'].astype(str) \
        + "_" + df_train['os'].astype(str)).apply(hash) % D
    click_buffer= np.full(D, 3000000000, dtype=np.uint32)

    df_train['epochtime'] = df_train['click_time'].astype(np.int64) // 10 ** 9
    next_clicks= []
    for category, t in zip(reversed(df_train['category'].values), reversed(df_train['epochtime'].values)):
        next_clicks.append(click_buffer[category]-t)
        click_buffer[category]= t
    del(click_buffer)
    qq = list(reversed(next_clicks))

    df_train.drop(['category'], axis=1, inplace=True)
    df_train['nextClick'] = pd.Series(qq).astype('float32')
    predictors.append('nextClick')

    print('[2/2] Extracting nextClick_sameChan')
    D=2**26
    df_train['category'] = (df_train['ip'].astype(str) + "_" + df_train['channel'].astype(str) + "_" + df_train['device'].astype(str) \
        + "_" + df_train['os'].astype(str)).apply(hash) % D
    click_buffer= np.full(D, 3000000000, dtype=np.uint32)

    next_clicks= []
    for category, t in zip(reversed(df_train['category'].values), reversed(df_train['epochtime'].values)):
        next_clicks.append(click_buffer[category]-t)
        click_buffer[category]= t
    del(click_buffer)
    qq = list(reversed(next_clicks))

    df_train.drop(['category'], axis=1, inplace=True)
    df_train['nextClick_sameChan'] = pd.Series(qq).astype('float32')
    predictors.append('nextClick_sameChan')

    return df_train, predictors

## Parameters

In [15]:
## Data stored in Kaggle
#TRAIN_PATH = ".kaggle/competitions/talkingdata-adtracking-fraud-detection/train.csv"
#TEST_PATH = ".kaggle/competitions/talkingdata-adtracking-fraud-detection/test.csv"

## Data downloaded locally
TRAIN_PATH = "~/.kaggle/competitions/talkingdata-adtracking-fraud-detection/train.csv"
TEST_PATH = "~/.kaggle/competitions/talkingdata-adtracking-fraud-detection/test.csv"

dtypes = {
    'ip': 'uint32', 'app': 'uint16', 'device': 'uint16',
    'os': 'uint16', 'channel': 'uint16', 'is_attributed': 'uint8',
    'click_id': 'uint32',
}

debug = 2
limit_features = False

nrows=184903891-1
frm = 144903891
nchunk = 1000000 #2000000
val_size=int(0.33*nchunk); # debug == 2
df_val = None

if debug == 0:
    ## No cross-validation, all test data
    nchunk = 40000000; val_size = 5000000
    frm = 21500000 # day 2/4
elif debug == 1:
    ## With cross-validation
    nchunk = 5000000; frm = 85000000 # day-1
    val_size = 2000000; frm_val = 144903891

## Train data boundaries (fraction corresponding to nchunk size).
to = frm + nchunk
test_ofst = 0

## Main Loop

In [16]:
%%time
############################################################################################################
#        LOADING DATA
############################################################################################################
(df_train, test_df, len_train, sub) = load_datasets(
    frm, to, debug=debug,
    test_ofst=test_ofst, dtypes=dtypes
)

if debug == 1:
    print('************ Cross-validation: Loading data (#%d samp) ************'.format(val_size))
    len_train = len(df_train) - len(test_df)
    df_val = pd.read_csv(TRAIN_PATH, parse_dates=['click_time'], skiprows=range(1,frm_val),
                         nrows=val_size, dtype=dtypes,
                         usecols=['ip', 'app', 'device', 'os', 'channel', 'click_time', 'is_attributed'])
    df_train = df_train.append(df_val)

Loading train: #1000000
Loading test: #100000
CPU times: user 57.9 s, sys: 6.95 s, total: 1min 4s
Wall time: 1min 14s


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [17]:
%%time
############################################################################################################
#        FEATURE EXTRACTION (1/2: base stats)
############################################################################################################
df_train['hour'] = pd.to_datetime(df_train.click_time).dt.hour.astype('uint8')
df_train['day'] = pd.to_datetime(df_train.click_time).dt.day.astype('uint8')
df_train['minute'] = pd.to_datetime(df_train.click_time).dt.minute.astype('uint8')

categorical = ['app', 'device', 'os', 'channel', 'hour', 'day']
predictors = ['app', 'device', 'os', 'channel', 'hour', 'day']

df_train, predictors = extract_stats_features(df_train, predictors=predictors)

Extracting stats features...
Counting unique  app  by  ['ip'] ...
ip_uniq_app_count max value =  54
Counting unique  channel  by  ['ip'] ...
ip_uniq_chan_count max value =  93
Aggregating by  ['ip', 'app'] ...
ip_app_count max value =  1142
CPU times: user 3.4 s, sys: 90.6 ms, total: 3.49 s
Wall time: 3.53 s


In [18]:
%%time
############################################################################################################
#        FEATURE EXTRACTION (1/2: timserie stats)
############################################################################################################
print('[Feature: next click period]')
(df_train, predictors) = extract_timeserie_features(df_train, predictors)

df_train['minute'] = pd.to_datetime(df_train.click_time).dt.minute.astype('uint8')
prev_len = len(df_train)

[Feature: next click period]
Extracting Timserie features...
[1/2] Extracting nextClick
[2/2] Extracting nextClick_sameChan
CPU times: user 15.9 s, sys: 827 ms, total: 16.7 s
Wall time: 16.7 s


In [19]:
############################################################################################################
#        FEATURE SELECTION
############################################################################################################
if limit_features is True:
    predictors = ['app','channel', 'X3', 'X0', 'nextClick',
                  'os', 'nextClickPeriod', 'device', 'hour','day',
                  'nextClick_sameChan', 'ip_app_count']
    categorical = ['app','channel','os','device','hour','day']

In [20]:
%%time
############################################################################################################
#        TRAINING (1/3: Prepare data for LGB)
############################################################################################################
## Drop unnecessary columns
df_train.drop(['click_time'], axis=1, inplace=True)
df_train.drop(['epochtime'], axis=1, inplace=True)

## Convert types
df_train['ip_uniq_app_count'] = df_train['ip_uniq_app_count'].astype('uint16')
df_train['ip_app_count'] = df_train['ip_app_count'].astype('uint16')
df_train['ip_uniq_chan_count'] = df_train['ip_uniq_chan_count'].astype('uint16')

## Learning Parmeters: LGB
params = {
    'learning_rate': 0.05,
    #'is_unbalance': 'true', # replaced with scale_pos_weight argument
    'num_leaves': 15,  # 2^max_depth - 1
    'max_depth': 4,  # -1 means no limit
    'min_child_samples': 100,  # Minimum number of data need in a child(min_data_in_leaf)
    'max_bin': 100,  # Number of bucbketed bin for feature values
    'subsample': 0.7,  # Subsample ratio of the training instance.
    'subsample_freq': 1,  # frequence of subsample, <=0 means no enable
    'colsample_bytree': 0.9,  # Subsample ratio of columns when constructing each tree.
    'min_child_weight': 0,  # Minimum sum of instance weight(hessian) needed in a child(leaf)
    'scale_pos_weight': 50, # because training data is extremely unbalanced 
}

## Target columns
target = 'is_attributed'

## Categorical columns
categorical = ['app', 'device', 'os', 'channel', 'hour', 'day']

CPU times: user 75.8 ms, sys: 16.4 ms, total: 92.2 ms
Wall time: 95.5 ms


In [21]:
############################################################################################################
#        TRAINING (2/3: Train/Test/CV Split)
############################################################################################################
test_df = df_train[len_train:]

if debug == 1:
    df_val = df_train[-val_size:]
    df_train = df_train[:len_train]
else:
    df_val = df_train[(len_train-val_size):len_train]
    df_train = df_train[:(len_train-val_size)]

print(df_train.shape, df_val.shape, test_df.shape)
print('Predictors:  %s' % predictors)
print('Categorical: %s' % categorical)

(670000, 15) (330000, 15) (100000, 15)
Predictors:  ['app', 'device', 'os', 'channel', 'hour', 'day', 'ip_app_count', 'nextClick', 'nextClick_sameChan']
Categorical: ['app', 'device', 'os', 'channel', 'hour', 'day']


In [22]:
############################################################################################################
#        TRAINING (3/3: Train LGB Model)
############################################################################################################

### LGB Model

In [23]:
(bst, best_iteration, eval_score) = lgb_train_cv(
    params, df_train, df_val, predictors, target, 
    objective='binary', 
    metrics='auc',
    early_stopping_rounds=30,
    verbose_eval=True, 
    num_boost_round=1000,
    categorical_features=categorical
)

pred_val_1 = bst.predict(df_val[predictors], num_iteration=best_iteration)

## Roc score
score = roc_auc_score(df_val.is_attributed, pred_val_1)
print(score)

Preparing validation datasets


/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 30 rounds.
[10]	train's auc: 0.745747	valid's auc: 0.809269
[20]	train's auc: 0.884998	valid's auc: 0.879312
[30]	train's auc: 0.893823	valid's auc: 0.884725
Early stopping, best iteration is:
[1]	train's auc: 0.943103	valid's auc: 0.922824

Model Report
bst1.best_iteration:  1
auc: 0.9228239503331953
0.9228239503331954


### XGB Model

In [24]:
xgb_params = {} # use default ones

model = xgb_train_cv(
    xgb_params, df_train, df_val, predictors, target,
    objective='binary:logistic',
    early_stopping_rounds=25,
    num_boost_round=200,
    verbose_eval=5
)

## Measure XGB Performance on cross-validation data
import xgboost as xgb
dvalid = xgb.DMatrix(df_val[predictors], df_val.is_attributed)

## predictions on val
pred_val_2 = model.predict(dvalid, ntree_limit=model.best_ntree_limit)

## Roc score
score = roc_auc_score(df_val.is_attributed, pred_val_2)
print(score)

/usr/local/lib/python3.7/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.7/dist-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[0]	train-auc:0.920719	valid-auc:0.904687
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 25 rounds.
[5]	train-auc:0.944757	valid-auc:0.932568
[10]	train-auc:0.960443	valid-auc:0.951543
[15]	train-auc:0.964595	valid-auc:0.955915
[20]	train-auc:0.96746	valid-auc:0.956836
[25]	train-auc:0.971455	valid-auc:0.957737
[30]	train-auc:0.973931	valid-auc:0.959764
[35]	train-auc:0.976851	valid-auc:0.960948
[40]	train-auc:0.978953	valid-auc:0.961042
[45]	train-auc:0.98089	valid-auc:0.961023
[50]	train-auc:0.982447	valid-auc:0.960863
[55]	train-auc:0.983517	valid-auc:0.961188
[60]	train-auc:0.984673	valid-auc:0.961275
[65]	train-auc:0.985393	valid-auc:0.961668
[70]	train-auc:0.98676	valid-auc:0.961098
[75]	train-auc:0.9875	valid-auc:0.961213
[80]	train-auc:0.987978	valid-auc:0.960703
[85]	train-auc:0.988528	valid-auc:0.96037
Stopping. Best iteration:
[64]	train-auc:0.985372	valid-auc:0.961703

0.9617033369772972


### Catboost

In [25]:
import catboost as cb
import sklearn

cat_features_index = [0,1,2,3]

def auc(m, train, test):
    return (sklearn.metrics.roc_auc_score(y_train, m.predict_proba(train)[:,1]),
                sklearn.metrics.roc_auc_score(y_test,m.predict_proba(test)[:,1]))

model_catb = cb.CatBoostClassifier(
    eval_metric="AUC", depth=7, iterations=200,
    l2_leaf_reg=4,
    learning_rate=0.25,
    od_type='Iter', od_wait=20,
    one_hot_max_size=50
)

model_catb.fit(df_train[predictors], df_train[target], cat_features=cat_features_index)

pred_val_3 = model_catb.predict_proba(df_val[predictors])[:,1]
score = roc_auc_score(df_val[target], pred_val_3)
print(score)

0:	total: 1.16s	remaining: 3m 49s
1:	total: 2.59s	remaining: 4m 16s
2:	total: 3.92s	remaining: 4m 17s
3:	total: 4.97s	remaining: 4m 3s
4:	total: 6.31s	remaining: 4m 6s
5:	total: 7.33s	remaining: 3m 56s
6:	total: 8.31s	remaining: 3m 49s
7:	total: 9.59s	remaining: 3m 50s
8:	total: 10.9s	remaining: 3m 52s
9:	total: 12.4s	remaining: 3m 54s
10:	total: 14s	remaining: 4m
11:	total: 14.9s	remaining: 3m 53s
12:	total: 16s	remaining: 3m 50s
13:	total: 17.5s	remaining: 3m 52s
14:	total: 18.7s	remaining: 3m 50s
15:	total: 20.1s	remaining: 3m 50s
16:	total: 21.1s	remaining: 3m 47s
17:	total: 22s	remaining: 3m 42s
18:	total: 22.9s	remaining: 3m 37s
19:	total: 23.6s	remaining: 3m 32s
20:	total: 24.4s	remaining: 3m 27s
21:	total: 25.1s	remaining: 3m 23s
22:	total: 26s	remaining: 3m 19s
23:	total: 26.8s	remaining: 3m 16s
24:	total: 27.8s	remaining: 3m 14s
25:	total: 28.8s	remaining: 3m 12s
26:	total: 30s	remaining: 3m 12s
27:	total: 31.2s	remaining: 3m 11s
28:	total: 32.3s	remaining: 3m 10s
29:	total: 

## Stacking Models: final predictions

### on validation data

In [26]:
w1 = 1/5.
w2 = 2/5.
w3 = 2/5.

pred_stacked_cv = pred_val_1 * w1 + pred_val_2 * w2 + pred_val_3 * w3

score = roc_auc_score(df_val[target], pred_stacked_cv)
print(score)

0.9632014158114894


### on test data

In [27]:
pred_1 = bst.predict(test_df[predictors], num_iteration=best_iteration)

dtest = xgb.DMatrix(test_df[predictors])
pred_2 = model.predict(dtest, ntree_limit=model.best_ntree_limit)

pred_3 = model_catb.predict_proba(test_df[predictors])[:,1]

pred_stacked_test = pred_1 * w1 + pred_2 * w2 + pred_3 * w3

In [28]:
############################################################################################################
#        PREDICTIONS on TEST data.
############################################################################################################
fnames = {
    0: 'final-output.csv',
    1: 'out-dev-big.csv',
    2: 'out-dev-small.csv'
}

fname = fnames[debug]
print('Saving output to: {fname}'.format(fname=fname))
sub[target] = pred_stacked_test
sub.to_csv(fname, index=False, float_format='%.9f')

Saving output to: out-dev-small.csv


In [29]:
!ls 

catboost_info  out-dev-small.csv  td-frauddetection-001.ipynb
